In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:


headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) "
                         "Chrome/115.0 Safari/537.36"}

loader = WebBaseLoader(
    "https://www.gutenberg.org/files/84/84-h/84-h.htm",
    requests_kwargs={"headers": headers}
)

docs = loader.load()


In [13]:
splitter  =   RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=300)
splitted_docs_ori = splitter.split_documents(docs)
len(splitted_docs_ori)
splitted_docs = splitted_docs_ori[:21]

In [14]:
from langchain.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large")

C:\Users\yasha\AppData\Local\Temp\ipykernel_35592\2256179261.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="mxbai-embed-large")


In [ ]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(splitted_docs, embeddings)
vectorstoredb.save_local("faiss_index")


In [41]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model = "gemini-2.0-flash")

In [52]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """You are a knowledgeable literature assistant. 
    Use ONLY the provided context (which comes from classic literature texts such as Project Gutenberg) 
    to answer the question. 
    If the context does not contain the answer, say 'I don't know based on the given text.' 
    Do not use outside knowledge or make assumptions.

    <context>
    {context}
    </context>

    Question: {input}
    Answer:"""
)

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,)

document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template="You are a knowledgeable literature assistant. \n    Use ONLY the provided context (which comes from classic literature texts such as Project Gutenberg) \n    to answer the question. \n    If the context does not contain the answer, say 'I don't know based on the given text.' \n    Do not use outside knowledge or make assumptions.\n\n    <context>\n    {context}\n    </context>\n\n    Question: {input}\n    Answer:"), additional_kwargs={})])
| GoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=ChatGoogleGenerativeAI(model='m

In [53]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever, document_chain)

In [54]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017AC5DAF940>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template="You are a knowledgeable literature assistant. \n    Use ONLY the provided context (which comes from classic literature texts such a

In [58]:
question = "What is Robert Walton's assessment of his voyage's progress and the morale of his crew by July 7th, and what is his personal declaration regarding its ultimate success?"
response = retriever_chain.invoke({"input":question})

response["answer"]

"By July 7th, Robert Walton says that he is safe and well advanced on his voyage and in good spirits. He also states that his men are bold and firm of purpose. He makes no personal declaration regarding the voyage's ultimate success in this letter."

In [49]:
response

{'input': "Who is the recipient of Robert Walton's first letter, and from what city is it sent?",
 'context': [Document(id='961ada54-5eb6-4983-b333-792a1ba37190', metadata={'source': 'https://www.gutenberg.org/files/84/84-h/84-h.htm', 'title': 'Frankenstein | Project Gutenberg', 'language': 'en'}, page_content='Your affectionate brother,\r\nRobert Walton\r\n\n\n\nLetter 3\n\nTo Mrs. Saville, England.\n\n\r\nJuly 7th, 17—.\r\n\n\r\nMy dear Sister,\r\n\n\r\nI write a few lines in haste to say that I am safe—and well advanced on my\r\nvoyage. This letter will reach England by a merchantman now on its homeward\r\nvoyage from Archangel; more fortunate than I, who may not see my native land,\r\nperhaps, for many years. I am, however, in good spirits: my men are bold and\r\napparently firm of purpose, nor do the floating sheets of ice that continually\r\npass us, indicating the dangers of the region towards which we are advancing,\r\nappear to dismay them. We have already reached a very high 

In [51]:
response = retriever_chain.invoke({"input": "What is the main theme of the story?"})
print(response["answer"])


What does the author say about his education?
